# Functional programming

Rust's design is highly influenced by functional languages, and it has features which are not familiar for C/C++ developers. In this chapter, we will learn those features.

## Algebraic data types

Rust has C-like enum:

In [2]:
#[derive(Debug)]
enum IpType {
    V4,
    V6,
}

IpType::V4

V4

So we can define ip as an struct:

In [3]:
#[derive(Debug)]
struct Ip(IpType, &'static str);

Ip(IpType::V4, "127.0.0.1")

Ip(V4, "127.0.0.1")

But it is not ideal. we want to store 4 bytes for ip v4 and 16 bytes for ip v6. Rust enum can handle that, because enum variants are able to have fields like an struct.

In [4]:
#[derive(Debug)]
enum Ip {
    V4([u8; 4]),
    V6([u8; 16]),
}

let ip = Ip::V4([127, 0, 0, 1]);
ip

V4([127, 0, 0, 1])

Rust enums enables writing typesafe code. In this example we can't make a version 6 ip with only 4 bytes.

But how we can use such enums? Getting field from those doesn't work (and doesn't mean, really):

In [5]:
ip.0

Error: no field `0` on type `Ip`

## Pattern matching

We saw pattern matching in the memory safety chapter. It also works on enums:

In [6]:
fn show_ip(ip: &Ip) {
    match ip {
        Ip::V4(x) => {
            println!("{}.{}.{}.{}", x[0], x[1], x[2], x[3]);
        }
        Ip::V6(_) => todo!(),
    }
}

show_ip(&ip);

127.0.0.1


That is, enum variants with patterns in fields are patterns. A more complex example:

In [8]:
fn show_ip(ip: &Ip) {
    match ip {
        Ip::V4([127, _, _, _]) => println!("localhost"),
        Ip::V4([192, 168, x, y]) => println!("local network 192.168.{x}.{y}"),
        Ip::V4([a, b, c, d]) => println!("ip {a}.{b}.{c}.{d}"),        
        Ip::V6(_) => todo!(),
    }
}

show_ip(&Ip::V4([192, 168, 1, 1]));

local network 192.168.1.1


`enum` variants can also have named fields:

In [10]:
enum Message {
    Quit,
    Move { x: i32, y: i32 },
    Write(&'static str),
    ChangeColor(i32, i32, i32),
}

let message = Message::Move { x: 12, y: -33 };
match message {
    Message::Move { x: _, y: 0.. } => "move up",
    Message::Move { x: _, y: _ } => "move down",
    _ => "unhandled",
}

"move down"

`struct` is also pattern:

In [13]:
struct Human {
    name: &'static str,
    height: u32,
}

let hamid = Human { name: "hamid", height: 190 };
match hamid {
    Human { height: 300.., name: _ } => println!("invalid"),
    Human { height: 180.., name: some_ident } => println!("A tall human {some_ident}"),
    // here `name` is shortcut for `name: name`
    Human { height: _, name } => println!("A normal human {name}"),
};

A tall human hamid


Patterns that matches everything, like `_`, `ident`, `(a, b)`, ... can be used in `let`:

In [14]:
let Human { height, name } = hamid;
(height, name)

(190, "hamid")

Or in function arguments:

In [15]:
fn sum_of_array([a, b, c, d]: [i32; 4]) -> i32 {
    a + b + c + d
}

sum_of_array([1, 2, 3, 4])

10

Or in `for` loop:

In [17]:
let v = vec![1, 5, 10];
// `.enumerate` converts an iterator of X to an iterator of (index, X)
for (i, x) in v.iter().enumerate() {
    println!("{i}: {x}");
}

0: 1
1: 5
2: 10


()

`mut` is part of pattern, not part of `let`, so we can write this:

In [18]:
let (mut x, y) = (2, 5); // x is mutable, y is not
x += 4;
(y, x)

(5, 6)

Or even use it outside of `let`, for example in function arguments, for variables, or even `match`:

In [19]:
match Some(2) {
    Some(mut x) => {
        x += 2;
        x
    }
    None => 12,
}

4

Here we saw `Option` again. Did you know that `Option` itself is an `enum` and we can define ours equivalent?

In [21]:
#[derive(Debug)]
enum MyOption<T> {
    MySome(T),
    MyNone,
}

use MyOption::*;

let x: MyOption<i32> = MySome(2);
x

MySome(2)

`MyOption` is exactly equal to `Option` of standard library. It even supports null pointer niche optimization:

In [22]:
use std::mem::size_of;

(size_of::<Option<i64>>(), size_of::<MyOption<i64>>(), size_of::<Option<&i64>>(), size_of::<MyOption<&i64>>())

(16, 16, 8, 8)

So there is no additional support for `Option` variants in pattern matching. It just follows the `enum` rules.

Some functions and return types are annotated with `#[must_use]` (Similar to `[[nodiscard]]` in C++17), and compiler will warn us if we don't use them:

In [23]:
#[deny(warnings)]
fn foo() {
    let v = vec![1, 2, 3, 4];
    v.get(3); // this is suspicious
}

Error: unused return value of `core::slice::<impl [T]>::get` that must be used

We can silence the compiler by `let _ = `:

In [24]:
#[deny(warnings)]
fn foo() {
    let v = vec![1, 2, 3, 4];
    let _ = v.get(3); // We explicitly want to ignore the result
}

`_` in `let _ = ` isn't a variable name. It's just a pattern, which matches everything. To see its difference with normal bindings, we can use `DebugDrop`:

In [26]:
#[derive(Debug)]
struct DebugDrop(&'static str);
impl Drop for DebugDrop {
    fn drop(&mut self) {
        println!("{} has been dropped", self.0);
    }
}

{
    let x = DebugDrop("1");
    let _ = x; // does nothing. `x` is not moved into `_`.
    let _ = DebugDrop("2"); // dropped immediately, since `_` can't hold it.
    let mut y = DebugDrop("3");
    y = x; // 3 will be dropped, and 1 will be moved from `x` into `y`.
    println!("end of scope");
    // 1 will be dropped here.
};

2 has been dropped
3 has been dropped
end of scope
1 has been dropped


The fact that `_` doesn't move the value into itself can be useful. For example, this works: 

In [113]:
let tuple = (vec![1, 2], vec![3, 4]);
let (x, _) = tuple;
let (_, y) = tuple;
(x, y)

([1, 2], [3, 4])

But this doesn't:

In [115]:
let tuple = (vec![1, 2], vec![3, 4]);
let (x, _unused) = tuple;
let (_unused, y) = tuple;
(x, y)

Error: use of moved value: `tuple.0`

Error: use of moved value: `tuple.1`

## ADT implementation

Rust is a low level language, so it's important to know how things are stored inside memory, to achieve the best possible performance from it. Let's start with `struct`. `struct` exact layout is unspecified, but it guarantees that each field is aligned per it's type. So:

In [2]:
use std::mem::{size_of, align_of};

struct Foo {
    field1: u8,
    field2: i32,
}

(size_of::<Foo>(), align_of::<Foo>())

(8, 4)

Althought `Foo` can be stored in 5 bytes, Rust adds some padding to ensure alignment of `i32`, which is 4. Since it preserve alignment, we can safely create pointers to its fields:

In [6]:
{
    let foo = Foo { field1: 1, field2: 5 };
    let ref_to_field = &foo.field2;
    *ref_to_field
}

5

If we want to force it to use 5 bytes, we can use `#[repr(packed)]`:

In [9]:
#[repr(packed)]
struct FooPacked {
    field1: u8,
    field2: i32,
}

(size_of::<FooPacked>(), align_of::<FooPacked>())

(5, 1)

But we can't get reference to its fields:

In [8]:
{
    let foo = FooPacked { field1: 1, field2: 5 };
    let ref_to_field = &foo.field2;
    *ref_to_field
}

Error: reference to packed field is unaligned

 In Rust, unlike C, compiler is allowed to reorder fields to save some space. If you want C behavior, you can use `#[repr(C)]`:

In [11]:
#[repr(C)]
struct BarC {
    field1: u8,
    field2: i32,
    field3: u8,
}
// layout: | --padding 3--------------- | -field1 1- | --field2 4------------------------- | --padding 3------------ | -field3 1- |

struct BarRust {
    field1: u8,
    field2: i32,
    field3: u8,
}
// possible layout: | --padding 2-- | -field3 1- | -field1 1- | --field2 4------------------------- |

(size_of::<BarC>(), size_of::<BarRust>())

(12, 8)

for `enum` without fields, the layout is usually similar to a `u8`:

In [12]:
enum Seasons {
    Winter,
    Summer,
    Fall,
    Spring,
}

size_of::<Seasons>()

1

You can define binary representation of each variable, which enables `as` cast for them:

In [14]:
enum Seasons {
    Winter = 1,
    Summer = 2,
    Fall = 6,
    Spring = 20,
}

Seasons::Fall as i32

6

`enum` with fields is usually stored like a tagged union in C. You still have access to get pointer to each field, so it still needs alignment:

In [16]:
enum WithField {
    Variant1(u8),
    Variant2(u16, i32),
}
// possible layout for Variant 1: | tag 1 = 0 | padding 6                                  | field1 1 |
// possible layout for Variant 2: | tag 1 = 1 | padding 1 | field1 2      | field2 4                  |

(size_of::<WithField>(), align_of::<WithField>())

(8, 4)

Getting a pointer to enum fields is being done by pattern matching:

In [20]:
{
    let x: &Option<i32> = &Some(2);
    match x {
        Some(y) => {
            // y is a `&i32` pointer to option's field
            let z: i32 = *y;
            z + 3
        }
        None => 12,
    }
}

5

Here, `y` magically became a reference. In early versions of Rust, it needed some verbose syntax to make it compile, but now it just works.

## Exercise

There is a smart pointer in Rust, named `Cow` (Clone on write) which is defined in this way:
```Rust
pub enum Cow<'a, B: ?Sized + 'a>
where
    B: ToOwned,
{
    /// Borrowed data.
    Borrowed(&'a B),

    /// Owned data.
    Owned(<B as ToOwned>::Owned),
}
```

It is useful when you want to conditionally clone a referenced data. For example:

In [10]:
use std::borrow::Cow;

fn sum(value: &mut Cow<'_, [i32]>, a_rare_condition: bool) -> i32 {
    if a_rare_condition {
        value.to_mut().push(100);
    }
    value.iter().sum()
}

let mut x: Cow<'_, [i32]> = Cow::Borrowed(&[1, 2, 3]);

// No clone will happen here
println!("{}", sum(&mut x, false));
x

6


[1, 2, 3]

In [11]:
// Clone will happen here
println!("{}", sum(&mut x, true));
x

106


[1, 2, 3, 100]

Now we want a special version of `Cow` for arrays. As we saw above, `Cow` will clone the whole array in case of a single push or index change. You should implement `CowVec`:

```Rust
pub enum CowVec<'a, B: Clone + 'a>
{
    /// Borrowed slice.
    Borrowed(&'a [B]),

    /// Owned element.
    Owned(B),
    
    Splited {
        left: Box<CowVec<'a, B>>,
        right: Box<CowVec<'a, B>>,
    }
}
```

It should implement `Index` and `IndexMut` traits, with these conditions:
* Both in `O(log(size))`
* `IndexMut` should make at most one clone

There are better data structures for this problem (for example, keeping the underlying reference with a hashmap mask) but working with this tree defined with `enum` is part of the exercise.

## Iterators

An iterator, in C++, is a pointer-like object that we can `++` it until it reaches some other iterator, like `.end()` of a collection, and we iterate over something this way. In Rust, an iterator holds both start and end pointers, and provides a `.next` method which returns an `Option`, which is `None` if we reached the end, and `Some(next_elem)` otherwise:

In [30]:
{
    let v = vec![1, 2, 5];
    let mut v_iter = v.iter();
    let mut v_iter2 = v.iter();
    println!("{:?}", v_iter.next());
    println!("{:?}", v_iter2.next());
    println!("{:?}", v_iter.next());
    println!("{:?}", v_iter.next());
    println!("{:?}", v_iter.next());
    println!("{:?}", v_iter.next());
};

Some(1)
Some(1)
Some(2)
Some(5)
None
None


Since iterators track their end, it's impossible to move them past the end and create memory problems, which is common in C++. Another memory safety problem that happens in C++ due iterators is changing the underlying collection while iterating over it, which is prevented by Rust borrow checking rules.

In [3]:
{
    let mut v = vec![1, 2, 5];
    let mut v_iter = v.iter();
    println!("{:?}", v_iter.next());
    println!("{:?}", v_iter.next());
    v.clear();
    println!("{:?}", v_iter.next()); // iterator is now on the air
}

Error: cannot borrow `v` as mutable because it is also borrowed as immutable

We can implement `Iterator` for our own types. Let's make a type similar to ranges:

In [33]:
struct MyRange {
    start: u64,
    end: u64,
}

impl Iterator for MyRange {
    type Item = u64;
    fn next(&mut self) -> Option<u64> {
        if self.start >= self.end {
            return None;
        }
        self.start += 1;
        Some(self.start - 1)
    }
}

let mut r = MyRange { start: 5, end: 8 };
println!("{:?}", r.next());
println!("{:?}", r.next());
println!("{:?}", r.next());
println!("{:?}", r.next());
println!("{:?}", r.next());
(r.start, r.end)

Some(5)
Some(6)
Some(7)
None
None


(8, 8)

This allow us to call `for` on it:

In [35]:
let r = MyRange { start: 5, end: 8 };
for x in r {
    println!("{x}");
};

5
6
7


In addition to `for`, there are some default methods on `Iterator` trait which we can call. For example, `.count` will call `.next()` until it reaches the end, and reports the count:

In [36]:
MyRange { start: 5, end: 8 }.count()

3

Or `.collect` which can convert an iterator to a collection, like a vector, a hashmap, ...

In [39]:
let mut x: Vec<u64> = MyRange { start: 5, end: 8 }.collect();
x[1] += 10;
x

[5, 16, 7]

Or `.skip(n)` which calls `.next()` n times and then returns the remaining iterator:

In [41]:
let mut x = MyRange { start: 5, end: 10 }.skip(3);
x.next()

Some(8)

We can implement `Iterator` methods ourself to make them faster:

In [47]:
struct MyRangeFast {
    start: u64,
    end: u64,
}

impl Iterator for MyRangeFast {
    type Item = u64;
    fn next(&mut self) -> Option<u64> {
        if self.start >= self.end {
            return None;
        }
        self.start += 1;
        Some(self.start - 1)
    }

    fn count(self) -> usize {
        usize::try_from(self.end - self.start).unwrap()
    }
}

MyRangeFast {
    start: 0,
    end: 1_000_000_000_000_000_000,
}.count()

1000000000000000000

Calling this on `MyRange` can take years to execute. Since people are allowed to override these methods, there is no guarantee that they will work correct. In fact, our example doesn't works correct, and it will overflow if `start` is after the end. Integer overflow in Rust causes a panic in debug builds, and a two's complement result in release builds (it is not UB unlike in C/C++):

In [48]:
:preserve_vars_on_panic 1

Preserve vars on panic: true


In [49]:
MyRangeFast {
    start: 10,
    end: 5,
}.count()

thread '<unnamed>' panicked at 'attempt to subtract with overflow', src/lib.rs:83:25
stack backtrace:
   0: rust_begin_unwind
             at /rustc/4b91a6ea7258a947e59c6522cd5898e7c0a6a88f/library/std/src/panicking.rs:584:5
   1: core::panicking::panic_fmt
             at /rustc/4b91a6ea7258a947e59c6522cd5898e7c0a6a88f/library/core/src/panicking.rs:142:14
   2: core::panicking::panic
             at /rustc/4b91a6ea7258a947e59c6522cd5898e7c0a6a88f/library/core/src/panicking.rs:48:5
   3: <core::panic::unwind_safe::AssertUnwindSafe<F> as core::ops::function::FnOnce<()>>::call_once
   4: run_user_code_36
   5: evcxr::runtime::Runtime::run_loop
   6: evcxr::runtime::runtime_hook
   7: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


But the standard library does it right:

In [51]:
(0..1_000_000_000_000_000_000_u64).count()

1000000000000000000

In [52]:
(10..5).count()

0

There are another family of functions on `Iterator` trait, called iterator adapters. Let's start with `.filter`:

In [53]:
(0..100).filter(|x| x % 2 == 0).count()

50

It accepts a function which returns a boolean, and returns a new iterator, consists only of elements which are true. To better see it, we can collect it to `Vec`:

In [54]:
(0..100).filter(|x| x % 2 == 0).collect::<Vec<_>>()

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 94, 96, 98]

Another adapter is `.map`, which transforms the iterator:

In [56]:
(0..20).map(|x| (x / 2, x % 2)).collect::<Vec<_>>()

[(0, 0), (0, 1), (1, 0), (1, 1), (2, 0), (2, 1), (3, 0), (3, 1), (4, 0), (4, 1), (5, 0), (5, 1), (6, 0), (6, 1), (7, 0), (7, 1), (8, 0), (8, 1), (9, 0), (9, 1)]

We can chain the iterator adapters:

In [57]:
(0..20)
    .filter(|x| x % 2 == 0)
    .map(|x| (x / 5, x % 5))
    .collect::<Vec<_>>()

[(0, 0), (0, 2), (0, 4), (1, 1), (1, 3), (2, 0), (2, 2), (2, 4), (3, 1), (3, 3)]

Iterator adaptors are lazy and do nothing without someone who calls `.next` on them (which here is `.collect` or `.count`) we can see that by printing inside closures.

In [58]:
(0..5)
    .map(|x| {
        println!("{x}");
        x + 100
    })
    .collect::<Vec<_>>()

0
1
2
3
4


[100, 101, 102, 103, 104]

In [59]:
(0..5)
    .map(|x| {
        println!("{x}");
        x + 100
    })

Map { iter: 0..5 }

And there was no print! Lazy iterators are how Rust archive ergonomics and safety at the same time. Performance of iterator adapters (after optimization) is equal to manual for loops. To show that, let's see the assembly of this function:

In [63]:
pub fn foo(x: Vec<i32>) -> i32 {
    x
        .iter()
        .filter(|x| **x < 10)
        .map(|x| x * x)
        .sum::<i32>()
}

foo(vec![1, 2, 12, 3])

14

```
example::foo:
        addi    sp, sp, -16
        sd      ra, 8(sp)
        sd      s0, 0(sp)
        mv      a1, a0
        ld      a2, 16(a0)
        ld      a0, 0(a0)
        li      s0, 0
        beqz    a2, .LBB0_5
        slli    a2, a2, 2
        li      a3, 10
        mv      a4, a0
        j       .LBB0_3
.LBB0_2:
        addi    a2, a2, -4
        addi    a4, a4, 4
        beqz    a2, .LBB0_5
.LBB0_3:
        lw      a5, 0(a4)
        bge     a5, a3, .LBB0_2
        mulw    a5, a5, a5
        addw    s0, s0, a5
        j       .LBB0_2
.LBB0_5:
        ld      a1, 8(a1)
        beqz    a1, .LBB0_8
        slli    a1, a1, 2
        beqz    a1, .LBB0_8
        li      a2, 4
        call    __rust_dealloc@plt
.LBB0_8:
        mv      a0, s0
        ld      ra, 8(sp)
        ld      s0, 0(sp)
        addi    sp, sp, 16
        ret
```

This is risc-v, not x86, which IMO is easier to read. As you can see, there is no function call (except the one which deallocates the vector) and it is a really small and simple loop.

If Rust was not lazy (like how JS `.map` and ... is implemented), every adapter would be forced to allocate the iterator items as something like vector, which would be very slow. Now that it is lazy, it can even implement infinite iterators, in constant memory! For example, here we find the 100 first primes:

In [69]:
fn is_prime(x: i64) -> bool {
    if x < 2 {
        return false;
    }
    !(2..x).take_while(|i| i * i <= x).any(|i| x % i == 0)
}

{
    // prime_iterator is almost infinite (it has at least 10^15 elements), but
    // since iterators are lazy, we can store it.
    let prime_iterator = (2..).filter(|x| is_prime(*x));
    // here `.collect` without `.take` would loop until OOM
    prime_iterator.take(100).collect::<Vec<_>>()
}

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541]

`.map`, `.filter`, `.take`, `.take_while`, ... which don't call `.next` on iterator and just return a wrapper around it, and are lazy, are called iterator adapters. Functions like `.collect`, `.count`, `.any` which call `.next()` on it are called iterator consumers. Another iterator consumer which is parent of `.sum`, `.all`, `.any`, `.mult`, ... is `.reduce`:

In [76]:
fn gcd(x: i32, y: i32) -> i32 {
    if y == 0 { x } else { gcd(y, x % y) }
}

let v = vec![12, 18, 60];
let sum = v.iter().sum::<i32>();
let sum_reduce = v.iter().copied().reduce(|a, b| a + b);
let gcd_reduce = v.iter().copied().reduce(gcd);
(sum, sum_reduce, gcd_reduce)

(90, Some(90), Some(6))

`.reduce` returns `None` in case of empty iterator, which you can use `.unwrap_or` on it.

There are many iterator adapters in standard library, but there is even more in `itertools` crate.

In [2]:
:dep itertools = "0.10"

`itertools` defines a trait which is implemented for every type that implements iterator. When we import that trait, new methods will appear on each iterator, like `.interleave`:

In [3]:
use itertools::Itertools;

(100..105).interleave(1..5).collect::<Vec<_>>()

[100, 1, 101, 2, 102, 3, 103, 4, 104]

Unlike standard library adapters which are zero cost, some of `itertools` adapters do some heavy jobs and sacrifice performance for ergonomics. For example, `.unique` internally keeps a `HashMap` to remove duplicate elements:

In [5]:
// This is not the fastest possible code, but it is clean.
vec![1, 2, 5, 2, 10, 5, 7, 1].into_iter().unique().collect::<Vec<_>>()

[1, 2, 5, 10, 7]

In addition to `Itertools` trait, `itertools` has some more utility. For example, `iproduct!` macro can be used instead of nested `for` loops:

In [10]:
use itertools::iproduct;

for (a, b, c) in iproduct!(0..2, 3..6, 'a'..'c') {
    println!("{a} {b} {c}");
}
// or even use it without for
iproduct!(1..5, 1..5).map(|(x, y)| x * y).sum::<i32>()

0 3 a
0 3 b
0 4 a
0 4 b
0 5 a
0 5 b
1 3 a
1 3 b
1 4 a
1 4 b
1 5 a
1 5 b


100

## Closure

We have seen closures, functions that can capture the environment:

In [79]:
let mut environment = 0;
let s = (0..100)
    .filter(|x| {
        environment += x; // this is considered bad Rust code
        *x < 10
    })
    .sum::<i32>();
(environment, s)

(4950, 45)

But we don't know how to write a function that take closures as inputs. Let's start with function pointers. Function pointers stores a pointer to the beginning of the assembly instructions we are going to jump to it. They are very similar to C function pointers, but they are always non null and valid (point to assembly instructions of a function with correct signature). Function pointers doesn't have lifetime syntax, they are always `'static` (since assembly is valid for the entire of the program). Their syntax is `fn(arg1, arg2, ...) -> ret`:

In [81]:
(size_of::<fn(i32) -> u64>(), size_of::<Option<fn(i32, u64, &'static str)>>())

(8, 8)

We can take a function pointer as an input and call it.

In [82]:
fn call_it_three_times(f: fn(i32) -> i32) {
    println!("for 1 -> {}", f(1));
    println!("for 2 -> {}", f(2));
    println!("for 3 -> {}", f(3));
}

fn pow3(x: i32) -> i32 {
    x * x * x
}

call_it_three_times(pow3);

for 1 -> 1
for 2 -> 8
for 3 -> 27


We can also call it with closures that capture no data:

In [84]:
call_it_three_times(|x| x * 2 + 5);

for 1 -> 7
for 2 -> 9
for 3 -> 11


But not with closures that capture data:

In [85]:
let mut env = 0;
call_it_three_times(|x| {
    env += 1;
    env + x
});

Error: mismatched types

To see why a closure can't be converted to a function pointer, let's see how compiler compiles a code like this:

In [88]:
{
    let mut env = 0;
    let mut closure = || {
        env += 1;
        env
    };
    closure();
    closure();
    env
}

2

It will becomes roughly this:

In [91]:
{
    let mut env = 0;
    let mut closure = {
        struct Cl<'a> {
            captured_env: &'a mut i32,
        }
        impl Cl<'_> {
            fn call(&mut self) -> i32 {
                *self.captured_env += 1;
                *self.captured_env
            }
        }
        Cl { captured_env: &mut env }
    };
    closure.call();
    closure.call();
    env
}

2

So closure is not a primitive thing, and it is implemented on top of normal functions and structs. So it follows normal Rust ownership and borrowing rules. It also explains why we need `mut` in `let mut closure`.

Type of the closure is the hidden struct that compiler generates for us. We can see that by printing the size of closures:

In [92]:
use std::mem::size_of_val;
{
    let a: i32 = 5;
    let b: i32 = 10;
    let closure1 = || a + b; // captures two pointer
    let closure2 = move || a + b; // captures two `i32` values
    let some_bytes: [u8; 47] = [0; 47];
    let closure3 = move || some_bytes[7]; // captures 47 bytes by value
    (size_of_val(&closure1), size_of_val(&closure2), size_of_val(&closure3))
}

(16, 8, 47)

So type of each closure differs from other one, and we can't change it at runtime:

In [94]:
{
    let env = 3;
    let mut closure = |x| x + env;
    closure = |x| x + env + 2;
}

Error: mismatched types

In this example, even the captures were equal, but compiler generated two different structs for them. Note that it works for function pointers because they are all the same type:

In [97]:
{
    let mut fp: fn(i32) -> i32 = |x| x + 2;
    fp = |x| x + 5;
    fp(3)
}

8

But if closures have different types, how we can take one as input? With generics! Every closure implements some of `FnOnce`, `FnMut`, `Fn` trait. For understanding what are they, let's get back to the desugaring above:

In [ ]:
{
    let mut env = 0;
    let mut closure = {
        struct Cl<'a> {
            captured_env: &'a mut i32,
        }
        impl Cl<'_> {
            fn call(&mut self) -> i32 {
             // note ^^^ this `mut`
                *self.captured_env += 1;
                *self.captured_env
            }
        }
        Cl { captured_env: &mut env }
    };
    closure.call();
    closure.call();
    env
}

We can imagine at least three things there: `&self`, `&mut self`, `self`. If closure only reads from the environment, it needs `&self`, and it implements `Fn` trait:

In [98]:
{
    let v = vec![1, 2, 10];
    let closure = || v.len();
    closure()
}

3

If it writes to the environment, it needs `&mut self`, and it implements `FnMut`:

In [100]:
{
    let mut v = vec![1, 2, 10];
    let closure = || { // `mut` is needed here
        v.push(3);
        v.len()
    };
    closure()
}

Error: cannot borrow `closure` as mutable, as it is not declared as mutable

As compiler says, `closure()` needs a `&mut`, which needs `let mut closure`. The last class of closures, take the ownership of some part of environment. They take `self` by value and implement `FnOnce`:

In [101]:
{
    let mut v = vec![1, 2, 10];
    let closure = move || {
        drop(v);
    };
    closure();
    closure();
}

Error: use of moved value: `closure`

Everything that implements `Fn`, also implements `FnMut` and everything that implements `FnMut` also implements `FnOnce`. Function pointers also implements `Fn`. So we can accept a `impl FnMut` in our function and user can call it with a `Fn` closure or a function pointer. Let's write `call_three_times` again:

In [107]:
fn call_it_three_times(mut f: impl FnMut(i32) -> i32) {
                    // ^^^ needed
    println!("for 1 -> {}", f(1));
    println!("for 2 -> {}", f(2));
    println!("for 3 -> {}", f(3));
}

let mut env = 0;
call_it_three_times(|x| {
    env += 10;
    env + x
});

for 1 -> 11
for 2 -> 22
for 3 -> 33


`FnMut(i32) -> i32` is a syntax sugar for something like `FnMut<(i32,), Ret=i32>` which arguments are generic parameter of the trait, and return type is an associated type for the trait. We can also use `dyn Fn` if we need runtime polymorphism:

In [124]:
{
    let d = DebugDrop("hello");
    let mut v = vec![1, 2, 3];
    let array: [Box<dyn FnOnce() -> usize>; 3] = [
        // Box is needed, since `dyn Trait` is unsized.
        // If we used `Fn` or `FnMut` instead of `FnOnce`, we could use `&` or `&mut` instead of `Box`
        Box::new(|| 5),
        Box::new(move || {
            println!("{:?}", d);
            d.0.len()
        }),
        Box::new(|| {
            v.push(4);
            v.len()
        })
    ];

    for (i, f) in array.into_iter().enumerate() {
        println!("result of {i} is {}", f());
    }
};

result of 0 is 5
DebugDrop("hello")
hello has been dropped
result of 1 is 5
result of 2 is 4


Note that `Fn` doesn't guarantee anything about side effects and mutability. We can mutate things with interior mutability types like `Mutex`, and even without them, we can make side effects and mutable states with foreign resources, like files and network. Here is a `Fn` closure which is a counter:

In [137]:
fn call_fn(f: &impl Fn() -> usize) {
    // we have a immutable reference, but the result still changes:
    println!("{}, {}, {}", f(), f(), f())
}

use std::fs::{write, read_to_string};

write("/tmp/counter", "").unwrap();

call_fn(&|| {
    let mut x = read_to_string("/tmp/counter").unwrap();
    x += "0";
    write("/tmp/counter", &x);
    x.len()
});

1, 2, 3


But still it can prevent accidental misuses. The difference between `Fn` and `FnMut` is also important in thread safety. `Fn` can be shared between threads, since it only reads from the environment. For example here we need a function that wants to run a closure on 5 threads:

In [141]:
use std::thread;

In [154]:
fn run_on_5_threads(mut f: impl FnMut() + Send + Sync) {
    thread::scope(|s| {
        for i in 0..5 {
            let f = &mut f; // we want to capture f by reference, since we can't copy it
            s.spawn(move || {
                println!("thread {i} started");
                f();
                println!("thread {i} finished");
            });
        }
    });
}

Error: cannot borrow `f` as mutable more than once at a time

But it works with `Fn`:

In [156]:
fn run_on_5_threads(f: impl Fn() + Send + Sync) {
    thread::scope(|s| {
        for i in 0..5 {
            let f = &f; // we want to capture f by reference, since we can't copy it
            s.spawn(move || {
                println!("thread {i} started");
                f();
                println!("thread {i} finished");
            });
        }
    });
}

run_on_5_threads(|| println!("hello"));

thread 0 started
hello
thread 0 finished
thread 3 started
hello
thread 3 finished
thread 1 started
hello
thread 1 finished
thread 2 started
hello
thread 2 finished
thread 4 started
hello
thread 4 finished


Trying to mutate a variable inside that closure (thankfully) doesn't compile, since it is not thread safe:

In [157]:
let mut env = 0;
run_on_5_threads(|| {
    env += 1;
    println!("hello {env}");
});

Error: cannot assign to `env`, as it is a captured variable in a `Fn` closure

But it will work with a `Mutex`:

In [161]:
use std::sync::Mutex;

let env = Mutex::new(0);
run_on_5_threads(|| {
    let mut env = env.lock().unwrap();
    *env += 1;
    println!("hello {env}");
});
env.into_inner()

thread 0 started
thread 1 started
hello 1
thread 0 finished
hello 2
thread 1 finished
thread 3 started
thread 2 started
hello 3
thread 3 finished
hello 4
thread 2 finished
thread 4 started
hello 5
thread 4 finished


Ok(5)

Now that we know the magic behind closures, we can implement iterator adapters manually. Let's implement `filter`:

In [133]:
struct FilterWrapper<T, F, I>
where
    I: Iterator<Item = T>,
    F: FnMut(&T) -> bool,
{
    predicate: F,
    iterator: I,
}

fn my_filter<T, F, I>(iter: I, f: F) -> FilterWrapper<T, F, I>
where
    I: Iterator<Item = T>,
    F: FnMut(&T) -> bool,
{
    FilterWrapper {
        predicate: f,
        iterator: iter,
    }
}

impl<T, F, I> Iterator for FilterWrapper<T, F, I>
where
    I: Iterator<Item = T>,
    F: FnMut(&T) -> bool,
{
    type Item = T;
    fn next(&mut self) -> Option<T> {
        while let Some(x) = self.iterator.next() {
            if (self.predicate)(&x) {
                return Some(x);
            }
        }
        None
    }
}

my_filter(0..100, |x| is_prime(*x)).collect::<Vec<_>>()

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]

You can see how `my_filter` is lazy and doesn't do anything inside of itself. The actual work happens at `.next` of `FilterWrapper`.

## `iter_mut` and `into_iter`

We have seen `.iter` on vector, which capture elements by reference:

In [22]:
let v = vec![1, 2, 5];
for x_ref in v.iter() {
    let x = *x_ref;
    println!("{x}");
};

1
2
5


There is a similar `.iter_mut`, which capture elements by mutable reference:

In [24]:
let mut v = vec![1, 2, 5];
for x_ref_mut in v.iter_mut() {
    *x_ref_mut += 10;
}
v

[11, 12, 15]

And there is `.into_iter()`, which capture by value and consume the vector:

In [25]:
let mut v = vec![1, 2, 3];
let s = v.into_iter().sum::<i32>();
v

Error: borrow of moved value: `v`

Result of `.into_iter()` owns the remaining elements and drop them when we drop it. We can see that with `DebugDrop`:

In [30]:
{
    let v: Vec<DebugDrop> = ["a", "b", "c", "d", "e"].into_iter().map(DebugDrop).collect();
    let mut it = v.into_iter();
    let a = it.next().unwrap();
    drop(it.next().unwrap());
    let mut it_rev = it.rev(); // it is now moved
    let e = it_rev.next().unwrap();
    println!("now we drop the iterator");
    drop(it_rev);
    println!("and then e and a will go out of scope");
};

b has been dropped
now we drop the iterator
c has been dropped
d has been dropped
and then e and a will go out of scope
e has been dropped
a has been dropped


But `.iter()` and `.iter_mut()` are just a reference to the underlying slice, that is, a pointer and a length:

In [33]:
use std::mem::size_of_val;
{
    let v = vec![1, 2, 5];
    let it = v.iter();
    size_of_val(&it)
}

16

But `.into_iter()` is responsible for freeing the underlying memory, so it needs:
* A pointer to the allocation start
* Capacity of the vector which is length of allocation (Rust's free needs the length as well, for performance reasons)
* Pointer or offset of start, which is the position of iterator
* Pointer or offset of end, which is usually length of the vector, but can be changed by `.rev()`

So we will see 4 * pointer size from this:

In [34]:
{
    let v = vec![1, 2, 5];
    let it = v.into_iter();
    size_of_val(&it)
}

32